In [1]:
%matplotlib auto
from emuCreate import *
import numpy as np
import matplotlib.pyplot as plt
import time

Using matplotlib backend: Qt5Agg


In [2]:
designFile = 'design.txt'
simOutFile = 'output.txt'

## lets read in the whole file and subsample
design = np.loadtxt(designFile)
ysim = np.loadtxt(simOutFile)

print(np.shape(design),np.shape(ysim.transpose()))


(1000, 5) (1000, 1620)


In [6]:
# Create the emulator
numofSamples = 2000
gpuOn = 1
hatch(designFile, simOutFile, numofSamples, gpuOn)


These basis vectors explain 99.96112142325146% of the variance
SetupModel: Determined data sizes as follows: 

SetupModel: This is a simulator (eta) -only model

SetupModel: m=1000  (number of simulated data)

SetupModel: p=  5  (number of inputs)

SetupModel: pu= 15 (transformed response dimension)





In [11]:
# Write a test design
simData = readdata(designFile,simOutFile,15) # third arg is number of bases
params = setupModel([],simData)
mTest = 1000;
testx = np.random.rand(mTest, params.model.p)
testx = testx * simData['orig']['xrange'] + simData['orig']['xmin']
# Create user defined inputs file
np.savetxt('new_inputs.txt', testx)
## Users can give their own input file below and predict

start = time.time()

## run emu on test input
allout = emuFly('new_inputs.txt',ranSamples=0)

end = time.time()

print('time taken to generate '+str(np.shape(allout)[0])+' uncertain values is: '+str(end - start)+' secs')


These basis vectors explain 99.96112142325146% of the variance
SetupModel: Determined data sizes as follows: 

SetupModel: This is a simulator (eta) -only model

SetupModel: m=1000  (number of simulated data)

SetupModel: p=  5  (number of inputs)

SetupModel: pu= 15 (transformed response dimension)



These basis vectors explain 99.96112142325146% of the variance
SetupModel: Determined data sizes as follows: 

SetupModel: This is a simulator (eta) -only model

SetupModel: m=1000  (number of simulated data)

SetupModel: p=  5  (number of inputs)

SetupModel: pu= 15 (transformed response dimension)



Default initial burn-in period: first 25% of samples
time taken to generate 1000 uncertain values is: 86.07866191864014 secs


In [ ]:
import time

infile = 'design.txt'
outfile = 'ayan_emu_uncert.txt'
start = time.time()

## run emu on test input
allout = emuFly(infile,ranSamples=0)

end = time.time()
print('time taken to generate '+str(np.shape(allout)[0])+' uncertain values is: '+str(end - start)+' secs')

np.savetxt(outfile,allout)

In [ ]:
print(np.shape(allout))

In [ ]:
print(np.min(allout),np.max(allout))
variances = np.loadtxt('yvar.dat')
#print((variances))

In [ ]:
infile = 'design.txt'
simIn = np.loadtxt(infile)
print(np.min(simIn,axis=0))
print(np.max(simIn,axis=0))
variances = np.loadtxt('yvar.dat')
print(np.shape(variances))

In [ ]:
variances = np.loadtxt('yvar.dat')
print(np.shape(variances))

In [ ]:
from sklearn.decomposition import PCA
import matplotlib as mpl

# PCA for the inputs 
pca2 = PCA(n_components=2)
# scale between 0 and 1 columswise
#simIn = (simIn - simIn.min(0)) / simIn.ptp(0)
#simIn = readSimIns(infile)
pcaCoords2 = pca2.fit_transform((simIn - simIn.min(0)) / simIn.ptp(0))

fig = plt.figure()

plt.scatter(pcaCoords2[:,0],pcaCoords2[:,1],c=variances, cmap=plt.cm.Spectral)

ax1 = fig.add_axes([0.92, 0.05, 0.02, 0.9])
cmap = mpl.cm.Spectral#mpl.cm.cool
norm = mpl.colors.Normalize(vmin=np.min(variances), vmax=np.max(variances))
cb1 = mpl.colorbar.ColorbarBase(ax1, cmap=cmap,
                                norm=norm,
                                orientation='vertical')
cb1.set_label('Variances')

plt.show()

In [ ]:
# try grid data
import matplotlib.pyplot as plt
from scipy.interpolate import griddata


grid_x, grid_y = np.mgrid[0:1:100j, 0:1:100j]
points = pcaCoords2#np.random.rand(1000, 2)
values = variances#func(points[:,0], points[:,1])


grid_z0 = griddata(points, values, (grid_x, grid_y), method='nearest')
grid_z1 = griddata(points, values, (grid_x, grid_y), method='linear')
grid_z2 = griddata(points, values, (grid_x, grid_y), method='cubic')
ext = (np.min(pcaCoords2[:,0]),np.max(pcaCoords2[:,0]),np.min(pcaCoords2[:,1]),np.max(pcaCoords2[:,1]))

fig = plt.figure()
plt.clf()
plt.subplot(221)
#plt.imshow(func(grid_x, grid_y).T, extent=(0,1,0,1), origin='lower')
plt.scatter(points[:,0], points[:,1], c=variances)
plt.title('Original')
plt.subplot(222)
plt.imshow(grid_z0.T, extent=ext, origin='lower')
plt.title('Nearest')
plt.subplot(223)
plt.imshow(grid_z1.T, extent=ext, origin='lower')
plt.title('Linear')
plt.subplot(224)
plt.imshow(grid_z2.T, extent=ext, origin='lower')
plt.title('Cubic')
#plt.gcf().set_size_inches(6, 6)
plt.show()

In [ ]:
pcaMin = np.min(pcaCoords2,axis=0)
pcaMax = np.max(pcaCoords2,axis=0)
print(pcaMin,pcaMax)
nx, ny = (50, 50)
x = np.linspace(pcaMin[0], pcaMax[0], nx)
y = np.linspace(pcaMin[1], pcaMax[1], ny)   
xv, yv = np.meshgrid(x, y)

In [ ]:
#print(x,y)
#print(np.shape(xv),np.shape(yv))
#print(xv,yv)

elems = []
import itertools
for element in itertools.product(x,y):
    elems.append(element)
elems = np.array(elems)
print(np.shape(elems))

In [ ]:
elems[-1,:]

In [ ]:
vals = pca2.inverse_transform(elems)
#vals = (pca2.inverse_transform(elems)+simIn.min(0))*simIn.ptp(0)

In [ ]:
print(vals)

In [ ]:
scVals = (simIn - simIn.min(0)) / simIn.ptp(0)
print(np.min(scVals,axis=0))
print(np.max(scVals,axis=0))

print(np.min(simIn,axis=0))
print(np.max(simIn,axis=0))
print(np.min(vals,axis=0))
print(np.max(vals,axis=0))